In [ ]:
import numpy as np
import h5py
import ccfj
import matplotlib.pyplot as plt
import yaml
import os

In [ ]:
import sys
sys.path.append(r'../')
from toollib_standard import mathlib
from toollib_standard import plotlib

In [ ]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
proj_name = proj['name']
proj_name = 'project/output_FJSJ_16-03/'
proj_name

In [ ]:
f_bands = [0,2,4,6,8,10,20,40]

In [ ]:
filename = proj_name+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = proj_name+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_stack = info_basic['dir_stack']
t = info_basic_bi['t']
f = info_basic_bi['f']
dt = 1/np.max(f)
t = (np.linspace(-len(f)-1,len(f)-1,2*(len(f)-1))+0.5)*dt/2

In [ ]:
key_subworks = info_basic['key_subworks']
#key_subworks.remove('23-08')
#key_subworks = ['07-03']
key_subworks

In [ ]:
key_subwork = key_subworks[0]
outname = key_subwork+'_gather_linear.h5'
ncffile = h5py.File(dir_stack + outname,'r')
ncfs = ncffile['ncfs'][:]
ncfst_linear = mathlib.freq_time(ncfs)
r = ncffile['r'][:]
ncffile.close()

In [ ]:
ncfs_sub = {}
for i in range(len(f_bands)-1):
    ncfs_sub[i] = np.zeros(np.shape(ncfs),dtype = complex)
    ncfs_sub[i][:,np.where((f>=f_bands[i])&(f<=f_bands[i+1]))] = ncfs[:,np.where((f>=f_bands[i])&(f<=f_bands[i+1]))]

In [ ]:
fig,ax = plt.subplots(len(f_bands),1,figsize = (15,5*len(f_bands)))
start = info_basic['start']
interval = info_basic['interval']
flag_time = info_basic['flag_time']
xlim_T = [-2,2]
flag_this = 0
title = 'all'
ax[flag_this] = plotlib.plot_ncfst(ax[flag_this],t,ncfst_linear[start::interval],r[start::interval],title,flag_time,xlim_T,0)
for i in range(len(f_bands)-1):
    flag_this += 1
    title = 'frequency band '+str(f_bands[i]) + '-' + str(f_bands[i+1]) + ' Hz'
    ncfst_sub = mathlib.freq_time(ncfs_sub[i])
    ax[flag_this] = plotlib.plot_ncfst(ax[flag_this],t,ncfst_sub[start::interval],r[start::interval],title,flag_time,xlim_T,0)

In [ ]:
def time_window_filter(t,ncfst0,r,v_min,t0,a):
    
    ncfst = ncfst0.copy()
    for i in range(len(ncfst)):
        tag = r[i]/v_min
        #print(t0,tag)
        t1 = t[t>-tag-t0][t[t>-tag-t0]< tag+t0]
        start = np.where(t == t1[0])[0][0]
        end = np.where(t == t1[-1])[0][0]
        ncfst[i][start:end+1]= ncfst[i][start:end+1]* np.exp(-a*np.abs((tag-np.abs(t1))))
    return ncfst

In [ ]:
ncfst_remove_sub = {}

In [ ]:
v_tag = 100000
t0 = 0.1
a = 50
for i in range(len(ncfs_sub)):
    ncfst_sub = mathlib.freq_time(ncfs_sub[i])
    ncfst_remove_sub[i] = time_window_filter(t,ncfst_sub,r,v_tag,t0,a)

In [ ]:
fig,ax = plt.subplots(len(f_bands)-1,2,figsize = (15,5*(len(f_bands))-1))
start = info_basic['start']
interval = info_basic['interval']
flag_time = info_basic['flag_time']
xlim_T = [-4,4]
flag_this = 0
title = 'all'
#ax[0][flag_this] = plotlib.plot_ncfst(ax[0][flag_this],t,ncfst_linear[start::interval],r[start::interval],title,flag_time,xlim_T,0)
for i in range(len(f_bands)-1):
    title = 'frequency band '+str(f_bands[i]) + '-' + str(f_bands[i+1]) + ' Hz'
    ncfst_sub = mathlib.freq_time(ncfs_sub[i])
    ax[flag_this][0] = plotlib.plot_ncfst(ax[flag_this][0],t,ncfst_sub[start::interval],r[start::interval],title,flag_time,xlim_T,0)
    title = 'remove frequency band '+str(f_bands[i]) + '-' + str(f_bands[i+1]) + ' Hz'
    ax[flag_this][1] = plotlib.plot_ncfst(ax[flag_this][1],t,ncfst_remove_sub[i][start::interval],r[start::interval],title,flag_time,xlim_T,0)
    flag_this += 1
    

### Gaussian time window

In [ ]:
def time_window_filter(t,ncfst0,r,v_min,t0,a):
    
    ncfst = ncfst0.copy()
    for i in range(len(ncfst)):
        tag = r[i]/v_min
        #print(t0,tag)
        t1 = t[t>-tag-t0][t[t>-tag-t0]< tag+t0]
        start = np.where(t == t1[0])[0][0]
        end = np.where(t == t1[-1])[0][0]
        ncfst[i][start:end+1]= ncfst[i][start:end+1]* np.exp(-a*np.abs((tag-np.abs(t1))))
    return ncfst

In [ ]:
def remove_zorocor(v_tag,t0,a,proj_name,flag_save,key_subwork_sample = []):
    filename = proj_name+'Basic_info.yml'
    with open(filename, 'r', encoding='utf-8') as f:
        info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
    filename_bi = proj_name+'Basic_info.npy'
    info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary
    key_subworks = info_basic['key_subworks']
    #key_subworks = ['08-10']

    flag_this = 0
    flag_all = len(key_subworks)
    

    for key_subwork in key_subworks:
        #v_tag = info_basic_bi['v_min'][key_subwork]
        #print(key_subwork)
        flag_this += 1
        dir_stack = info_basic['dir_stack']
        #t = info_basic_bi['t']
        f = info_basic_bi['f']
        dt = 1/np.max(f)
        t = np.linspace(-len(f)-1,len(f)-1,2*(len(f)-1))*dt/2
        t = (np.linspace(-len(f)-1,len(f)-1,2*(len(f)-1))+0.5)*dt/2

        inname = key_subwork+'_gather_linear.h5'
        outname = key_subwork+'_gather_timewindow.h5'
        ncffile_in = h5py.File(dir_stack + inname,'r')
        if os.path.exists(dir_stack+outname):
            os.remove(dir_stack+outname)
        ncffile = h5py.File(dir_stack+outname,'w')

        #print(ncffile.keys())

        ncfs_sum_linear = ncffile_in['ncfs'][:]
        r = ncffile_in['r'][:]

        ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
        ncfst1 = time_window_filter(t,ncfst_linear,r,v_tag,t0,a)
        ncfs_sum_remove = mathlib.time_freq(ncfst1)

        

        if flag_save:
            #print('save')
            ncffile.create_dataset('ncfs', data=ncfs_sum_remove) 
            ncffile.create_dataset('r',data=r)
            ncffile.close()  
        #ncffile.create_dataset('r', data=r)
        else:
            print('unsave')
            #print(ncffile.keys())
            ncffile.close()    
            
            return ncfs_sum_remove
        
        print(key_subwork+' is done! ' + str(flag_this) + '/' + str(flag_all))
            

In [ ]:
v_tag = 0.65
a = 10
t0 = 0.0
#tag = 0.00055
#a = 0.5
key_subwork_sample = [key_subworks[0]]
ncfs_sum_linear_remove = remove_zorocor(v_tag,t0,a,proj_name,0,key_subwork_sample)

In [ ]:
info_basic['v_tag'] = v_tag
info_basic['t0'] = t0
with open(proj_name+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [ ]:
key_subwork = key_subwork_sample[0]
outname = key_subwork+'_gather_linear.h5'
ncffile = h5py.File(dir_stack + outname,'r')
#ncfs_sum_linear_remove = ncffile['ncfs_remove'][:]
ncfst_linear_remove = mathlib.freq_time(ncfs_sum_linear_remove)
ncfs_sum_linear = ncffile['ncfs'][:]
ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
r = ncffile['r'][:]
ncffile.close()

In [ ]:
fig,ax = plt.subplots(1,2,figsize = (20,10))

start = info_basic['start']
interval = info_basic['interval']
flag_time = info_basic['flag_time']
xlim_T = [-1,1]
title1 = 'Linear'
ax[0] = plotlib.plot_ncfst(ax[0],t,ncfst_linear[start::interval],r[start::interval],title1,flag_time,xlim_T,0)
#ax[0].vlines(tag,0,0.2,linestyles='dashed',colors='g')
#ax[0].vlines(-tag,0,0.2,linestyles='dashed',colors='g')
ax[1] = plotlib.plot_ncfst(ax[1],t,ncfst_linear_remove[start::interval],r[start::interval],title1,flag_time,xlim_T,0)
#ax[1].vlines(tag,0,0.2,linestyles='dashed',colors='g')
#ax[1].vlines(-tag,0,0.2,linestyles='dashed',colors='g')
for i in range(0,len(r),interval):
    ax[0].plot(t0+r[i]/v_tag,r[i],'r.')
    ax[0].plot(-t0-r[i]/v_tag,r[i],'r.')
    ax[1].plot(t0+r[i]/v_tag,r[i],'r.')
    ax[1].plot(-t0-r[i]/v_tag,r[i],'r.')
plt.tight_layout()
plt.show()

In [ ]:
remove_zorocor(v_tag,t0,a,proj_name,1)